In [ ]:
# Imports
#pip install streamlit supabase osmnx geopandas rasterio shapely pyproj pydeck
# all possibles so far, may need to add more as necessary
import streamlit as st
from supabase import create_client, Client
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import tempfile
import rasterio
from rasterio.mask import mask
import osmnx as ox
from shapely.geometry import Polygon, mapping
import pydeck as pdk
import json
from datetime import datetime, timedelta

In [ ]:
# SQL code for Supabase table - run only once!
create table if not exists building_suitability (
  id serial primary key,
  city text,
  created_at timestamptz default now(),
  geojson jsonb
);

In [ ]:
# app.py
st.set_page_config(layout="wide", page_title="Solar Suitability Planner")

# -----------------------
# Supabase init (secrets)
# -----------------------
url = st.secrets["SUPABASE_URL"]
key = st.secrets["SUPABASE_KEY"]
supabase: Client = create_client(url, key)

# -----------------------
# Constants / defaults
# -----------------------
# Default average daily insolation in kWh/m2/day (tweakable)
DEFAULT_INSOLATION = {
    "Ann Arbor": 4.0,   # kWh/m2/day (approx; user may replace with better data)
    "Tucson": 6.0,
}
PANEL_EFFICIENCY = 0.18   # 18%
PERFORMANCE_RATIO = 0.75  # losses (inverter, soiling, wiring)

In [ ]:
# -----------------------
# Utility functions
# -----------------------
@st.cache_data(show_spinner=False)
def load_demand_table(city: str) -> pd.DataFrame:
    """
    Load city demand table from Supabase. Adjust table names if different.
    Expected numeric column: MW and datetime column named 'datetime'.
    """
    table_name = "Ann_Arbor_demand" if city == "Ann Arbor" else "TEPC_demand"
    res = supabase.table(table_name).select("*").execute()
    data = res.data
    df = pd.DataFrame(data)
    return df

In [ ]:
def download_geotiff_from_supabase(bucket: str, path: str) -> rasterio.io.DatasetReader:
    """
    Download a file from Supabase storage bucket to a temp file and open with rasterio.
    path is the object path (e.g., "satellite/ann_arbor.tif").
    """
    # supabase-py storage.download returns bytes (or a response-like object) depending on client
    res = supabase.storage.from_(bucket).download(path)
    # The client sometimes returns bytes or an object with .content
    if isinstance(res, (bytes, bytearray)):
        b = res
    elif hasattr(res, "content"):
        b = res.content
    else:
        # attempt to string-encode
        b = bytes(res)
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(path)[1] or ".tif")
    tmp.write(b)
    tmp.flush()
    tmp.close()
    raster = rasterio.open(tmp.name)
    return raster

In [ ]:
def fetch_buildings_osm(place_name: str) -> gpd.GeoDataFrame:
    """
    Use OSMnx to fetch building footprints for the given place name.
    Returns GeoDataFrame in EPSG:4326 with area_m2 computed.
    """
    tags = {"building": True}
    gdf = ox.geometries_from_place(place_name, tags=tags)
    # Only polygons (some OSM features are points/lines)
    gdf = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()
    # Reproject to metric to compute area in m^2
    gdf = gdf.to_crs(epsg=3857)
    gdf["area_m2"] = gdf.geometry.area
    # back to WGS84
    gdf = gdf.to_crs(epsg=4326)
    # basic commercial flag from tags if present (fallback False)
    gdf["is_commercial"] = gdf.apply(
        lambda row: (
            (pd.notna(row.get("building")) and str(row.get("building")).lower() in
             ["commercial", "retail", "industrial", "office", "warehouse"])
        ), axis=1
    )
    return gdf

In [ ]:
def polygon_orientation(polygon: Polygon):
    """
    Estimate roof orientation (degrees clockwise from North) by minimum_rotated_rectangle method.
    Returns degrees 0-360 where 0 = East? We'll return as degrees clockwise from North:
    convert from arctan2(dx, dy) with adjustments.
    """
    if polygon.is_empty:
        return np.nan
    mrr = polygon.minimum_rotated_rectangle
    coords = list(mrr.exterior.coords)
    max_len = 0
    best_angle = None
    # iterate edges
    for i in range(len(coords)-1):
        x1,y1 = coords[i]
        x2,y2 = coords[i+1]
        dx = x2 - x1
        dy = y2 - y1
        edge_len = np.hypot(dx, dy)
        if edge_len > max_len:
            max_len = edge_len
            best_angle = np.degrees(np.arctan2(dy, dx))
    if best_angle is None:
        return np.nan
    # best_angle is angle from +x axis (East) CCW. Convert to degrees clockwise from North:
    # angle_from_north_clockwise = (90 - best_angle) mod 360
    angle = (90 - best_angle) % 360
    return angle

In [ ]:
def orientation_match_score(roof_angle_deg: float, ideal_sun_azimuth_deg: float):
    """
    Compute orientation match score 0-1 where 1 = perfect face to sun,
    using smallest absolute difference on circle (0..180)
    """
    diff = abs(roof_angle_deg - ideal_sun_azimuth_deg) % 360
    if diff > 180:
        diff = 360 - diff
    # map 0->1, 180->0
    return 1.0 - (diff / 180.0)

In [ ]:
def compute_city_annual_kwh(df: pd.DataFrame):
    """
    Convert the 15-minute MW measurements into annual kWh.
    Each MW measurement over a 15-min interval contributes: MW * 0.25 hours = MWh.
    Sum of (MW) * 0.25 gives MWh per dataset period; multiply by 1000 to kWh.
    """
    total_mw_sum = df["MW"].sum()  # sum of instantaneous MW over all rows
    # energy_mwh = total_mw_sum * 0.25
    energy_mwh = total_mw_sum * 0.25
    annual_kwh = energy_mwh * 1000.0
    return annual_kwh

In [ ]:
def estimate_tilt_from_height_and_span(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Estimate roof tilt in degrees using any available height / building:levels or fallback heuristics.

    - If 'height' available (meters), use it
    - Else if 'building:levels' available, multiply by 3.2
    - Else fallback to small heights with default tilt for commercial vs residential
    """
    # handle numeric heights
    if "height" in gdf.columns:
        heights = pd.to_numeric(gdf["height"], errors="coerce")
    elif "building:levels" in gdf.columns:
        heights = pd.to_numeric(gdf["building:levels"], errors="coerce") * 3.2
    else:
        heights = pd.Series(np.nan, index=gdf.index)

    # Effective roof span (approx): use sqrt(area) or MRR longest edge-derived width
    def approx_width(geom):
        try:
            mrr = geom.minimum_rotated_rectangle
            # approximate width by dividing perimeter by 4 (rectangle assumption)
            perim = mrr.length
            width = perim / 4.0
            return max(width, np.sqrt(geom.area))  # safe fallback
        except Exception:
            return np.sqrt(geom.area)

    gdf["width_m"] = gdf.geometry.to_crs(epsg=3857).apply(lambda geom: approx_width(geom))
    # assign heights fallback
    heights_filled = heights.fillna(5.0)  # 5m fallback
    # tilt = arctan(height / (width/2)) [approx roof pitch]
    gdf["tilt_deg"] = np.degrees(np.arctan2(heights_filled, gdf["width_m"]/2.0))
    # clamp sensible range
    gdf["tilt_deg"] = gdf["tilt_deg"].clip(3, 45)
    return gdf

In [ ]:
def shade_from_geotiff(raster: rasterio.io.DatasetReader, polygon: Polygon):
    """
    Compute a simple shade score (0 bright / 1 shaded) by masking the raster to the polygon and computing mean brightness.
    Assumes raster band order such that first band is usable (grayscale or visible).
    """
    try:
        # ensure geometry in same CRS as raster (raster is likely in EPSG:4326 or other)
        # rasterio.mask expects geometries in same CRS; our polygon is in EPSG:4326
        out_img, out_transform = mask(raster, [mapping(polygon)], crop=True)
        arr = out_img[0].astype(float)
        # if completely masked, return 0
        if arr.size == 0:
            return 0.0
        # normalize
        arr_min, arr_max = np.nanmin(arr), np.nanmax(arr)
        if arr_max - arr_min < 1e-6:
            mean_norm = 0.5
        else:
            mean_norm = (np.nanmean(arr) - arr_min) / (arr_max - arr_min)
        # darker pixel -> higher shade score
        shade = 1.0 - mean_norm
        # clamp
        return float(np.clip(shade, 0.0, 1.0))
    except Exception:
        return 0.0

In [ ]:
def compute_suitability_scores(gdf: gpd.GeoDataFrame, irradiance_factor: float, ideal_sun_azimuth: float):
    """
    Compose final solar_score from orientation, tilt, shade, irradiance and area.
    We compute intermediate scores in [0,1] and combine with weights.
    """
    # orientation
    gdf["orientation_deg"] = gdf.geometry.apply(lambda geom: polygon_orientation(geom))
    gdf["orientation_score"] = gdf["orientation_deg"].apply(lambda a: orientation_match_score(a, ideal_sun_azimuth))
    # tilt: ideal ~25 deg (residential) -> score drops as further away
    gdf["tilt_score"] = 1.0 - (np.abs(gdf["tilt_deg"] - 25.0) / 40.0)
    gdf["tilt_score"] = gdf["tilt_score"].clip(0,1)
    # shade already 0..1 where 1 is shaded; we want exposure_score = 1 - shade
    gdf["exposure_score"] = (1.0 - gdf["shade"]).clip(0,1)
    # irradiance_factor applied uniformly (0..1)
    irr = float(np.clip(irradiance_factor, 0.0, 1.0))
    # combine: weights can be tuned
    gdf["solar_score"] = (
        0.35 * gdf["orientation_score"] +
        0.25 * gdf["tilt_score"] +
        0.25 * gdf["exposure_score"] +
        0.15 * irr
    )
    gdf["solar_score"] = gdf["solar_score"].clip(0,1)
    return gdf

In [ ]:
def estimate_building_annual_potential_kwh(gdf: gpd.GeoDataFrame, insolation_kwh_m2_day: float,
                                           panel_efficiency=PANEL_EFFICIENCY, perf_ratio=PERFORMANCE_RATIO):
    """
    Estimate the annual kWh each building can produce:
       potential_kwh = area_m2 * insolation_kwh_m2_day * 365 * efficiency * perf_ratio
    """
    daily = float(insolation_kwh_m2_day)
    factor = daily * 365.0 * panel_efficiency * perf_ratio
    gdf["annual_potential_kwh"] = gdf["area_m2"] * factor
    return gdf

In [ ]:
def select_buildings_to_meet_target(gdf: gpd.GeoDataFrame, required_kwh: float, commercial_pct: float):
    """
    Greedy selection that alternates choosing commercial/residential in a ratio
    that tries to match the requested commercial_pct while picking highest solar_score first.
    Returns GeoDataFrame of selected buildings and remaining totals.
    """
    # split pools
    comm = gdf[gdf["is_commercial"]==True].sort_values("solar_score", ascending=False).copy()
    resid = gdf[gdf["is_commercial"]==False].sort_values("solar_score", ascending=False).copy()

    # iteratively pick from pools to reach required_kwh
    sel_rows = []
    ptr_comm = 0
    ptr_resid = 0
    total_selected_kwh = 0.0
    selected_comm = 0
    selected_total = 0

    # Avoid division by zero: desired fraction [0..1]
    desired_comm_frac = commercial_pct / 100.0

    # Safety: if both pools empty, return empty
    while total_selected_kwh < required_kwh and (ptr_comm < len(comm) or ptr_resid < len(resid)):
        # choose which pool to pick from based on current fraction
        current_frac = (selected_comm / selected_total) if selected_total > 0 else 0.0
        pick_comm = False
        # If we have not yet reached desired commercial fraction and comm pool not empty -> pick commercial
        if current_frac < desired_comm_frac and ptr_comm < len(comm):
            pick_comm = True
        # else pick residential if available
        elif ptr_resid < len(resid):
            pick_comm = False
        # else fallback to commercial if residential exhausted
        elif ptr_comm < len(comm):
            pick_comm = True
        else:
            break

        if pick_comm:
            row = comm.iloc[ptr_comm]
            ptr_comm += 1
        else:
            row = resid.iloc[ptr_resid]
            ptr_resid += 1

        sel_rows.append(row)
        total_selected_kwh += float(row["annual_potential_kwh"])
        selected_total += 1
        if row["is_commercial"]:
            selected_comm += 1

    selected_gdf = gpd.GeoDataFrame(sel_rows).reset_index(drop=True) if len(sel_rows) > 0 else gpd.GeoDataFrame(columns=gdf.columns)
    # compute metrics
    actual_comm_pct = (selected_comm / selected_total * 100.0) if selected_total > 0 else 0.0
    return selected_gdf, total_selected_kwh, selected_total, actual_comm_pct

In [ ]:
# -----------------------
# Streamlit UI and main flow
# -----------------------
st.title("Solar Suitability Planner — Building Footprints & Model")
st.markdown("Compute building-level solar suitability and select parcels to meet a solar target.")

# Sidebar controls
city = st.sidebar.selectbox("City", ["Ann Arbor", "Tucson"])
solar_pct = st.sidebar.slider("Percent of city energy to meet with solar", 1, 100, 30)
commercial_pct = st.sidebar.slider("Percent of selected buildings to be commercial", 0, 100, 20)
insolation_override = st.sidebar.number_input("Insolation (kWh/m²/day) — optional override",
                                              value=float(DEFAULT_INSOLATION[city]), min_value=0.0, step=0.1)

In [ ]:
# Buttons
if st.sidebar.button("Run full model"):
    with st.spinner("Loading demand data from Supabase..."):
        df_demand = load_demand_table(city)
    if df_demand.empty:
        st.error("Loaded demand table is empty. Check Supabase table names and permissions.")
        st.stop()

    with st.spinner("Computing annual energy need..."):
        annual_kwh = compute_city_annual_kwh(df_demand)

    st.write(f"Estimated annual city energy consumption: **{annual_kwh:,.0f} kWh**")

    required_kwh = annual_kwh * (solar_pct / 100.0)
    st.write(f"Target annual solar generation: **{required_kwh:,.0f} kWh ({solar_pct}%)**")

    # Raster load (optional)
    # We expect satellite files stored under 'satellite/{city_filename}.tif' in bucket "satellite"
    tif_name = "ann_arbor.tif" if city == "Ann Arbor" else "tucson.tif"
    try:
        with st.spinner("Downloading GeoTIFF from Supabase..."):
            raster = download_geotiff_from_supabase("satellite", tif_name)
    except Exception as e:
        st.warning(f"Could not download GeoTIFF: {e}. Continuing without shade (shade=0).")
        raster = None

    with st.spinner("Fetching building footprints from OSM (this may take a minute)..."):
        place_name = f"{city}, USA"
        try:
            buildings = fetch_buildings_osm(place_name)
        except Exception as e:
            st.error(f"OSMnx fetch failed: {e}")
            st.stop()

    st.write(f"Fetched {len(buildings)} building footprints.")

    # compute geometry-derived features
    with st.spinner("Estimating roof geometry (orientation, tilt, area)..."):
        # orientation, tilt heuristics
        buildings["orientation_deg"] = buildings.geometry.apply(lambda g: polygon_orientation(g))
        buildings = estimate_tilt_from_height_and_span(buildings)

    # shade
    if raster is not None:
        st.info("Computing simple shade index from GeoTIFF per building (may take a while for many buildings)...")
        # apply shade (cautious about time; if many buildings, sample or parallelize)
        shade_vals = []
        for idx, row in stqdm := stqdm if "stqdm" in globals() else enumerate(buildings.itertuples()):  # safe fallback
            # above line builds a variable stqdm only if available; fallback enumerate
            break
        # We'll not rely on stqdm; do plain loop with progress
        progress = st.progress(0)
        total = len(buildings)
        shades = []
        for i, (idx, row) in enumerate(buildings.iterrows()):
            try:
                s = shade_from_geotiff(raster, row.geometry)
            except Exception:
                s = 0.0
            shades.append(s)
            if total > 0:
                progress.progress(int((i+1)/total * 100))
        buildings["shade"] = shades
        progress.empty()
    else:
        buildings["shade"] = 0.0

    # irradiance factor (normalize to 0..1) using daylight/azimuth proxies from demand df
    # We'll compute a simple irradiance_factor: map insolation value into 0..1 by dividing by 7 (approx max)
    irr_factor = np.clip(insolation_override / 7.0, 0.0, 1.0)
    # ideal sun azimuth: take mean of sunrise->sunset azimuths if available else 180 (south)
    if {"azimuth_sunrise","azimuth_sunset"}.issubset(df_demand.columns):
        try:
            mean_azimuth_range = ((df_demand["azimuth_sunset"] + df_demand["azimuth_sunrise"]) / 2.0).mean()
            ideal_azimuth = float(mean_azimuth_range)
        except Exception:
            ideal_azimuth = 180.0
    else:
        ideal_azimuth = 180.0

    with st.spinner("Computing solar suitability scores and annual potential..."):
        buildings = compute_suitability_scores(buildings, irr_factor, ideal_azimuth)
        buildings = estimate_building_annual_potential_kwh(buildings, insolation_override)

    # Show top candidates
    st.write("Top 10 candidate roofs by solar_score:")
    st.dataframe(buildings[["area_m2","is_commercial","orientation_deg","tilt_deg","shade","solar_score","annual_potential_kwh"]].sort_values("solar_score", ascending=False).head(10))

    # Selection algorithm
    with st.spinner("Selecting buildings to meet target while respecting commercial mix..."):
        selected_gdf, tot_kwh_sel, n_selected, actual_comm_pct = select_buildings_to_meet_target(buildings, required_kwh, commercial_pct)

    st.write(f"Selected {n_selected} buildings producing ~**{tot_kwh_sel:,.0f} kWh/year** (target was {required_kwh:,.0f} kWh).")
    st.write(f"Requested commercial%: **{commercial_pct}%** — actual selected commercial%: **{actual_comm_pct:.1f}%**")

    # Map visualization: PyDeck with two layers: all buildings choropleth, and selected highlighted
    # GeoJSON for all buildings colored by solar_score
    all_geojson = json.loads(buildings.to_json())
    selected_geojson = json.loads(selected_gdf.to_json()) if not selected_gdf.empty else {"type":"FeatureCollection","features":[]}

    # prepare color function in PyDeck: we will store color arrays in properties
    def score_to_color(score):
        # green (good) to red (bad) mapping — returns [r,g,b,a]
        g = int(np.clip(score,0,1) * 255)
        r = int((1 - np.clip(score,0,1)) * 255)
        return [r, g, 30, 140]

    # add colors as properties to geojson features
    for feat in all_geojson["features"]:
        score = feat["properties"].get("solar_score", 0.0)
        feat["properties"]["color"] = score_to_color(score)

    for feat in selected_geojson.get("features", []):
        # highlight selected with a bright blue
        feat["properties"]["color"] = [30, 144, 255, 200]

    # layers
    all_layer = pdk.Layer(
        "GeoJsonLayer",
        all_geojson,
        get_fill_color="properties.color",
        get_line_color=[80,80,80],
        pickable=True,
        stroked=True,
        filled=True,
        opacity=0.6,
    )
    sel_layer = pdk.Layer(
        "GeoJsonLayer",
        selected_geojson,
        get_fill_color="properties.color",
        get_line_color=[0,0,0],
        pickable=True,
        stroked=True,
        filled=True,
        opacity=0.9,
    )

    # compute center
    center = buildings.geometry.unary_union.centroid
    initial_view = pdk.ViewState(latitude=center.y, longitude=center.x, zoom=13, pitch=40)

    st.pydeck_chart(pdk.Deck(layers=[all_layer, sel_layer], initial_view_state=initial_view, map_style="mapbox://styles/mapbox/light-v9"))

    # Results summary
    st.header("Results")
    st.metric("City annual energy (kWh)", f"{annual_kwh:,.0f}")
    st.metric("Solar target (kWh/year)", f"{required_kwh:,.0f}")
    st.metric("Selected buildings (count)", f"{n_selected}")
    st.metric("Selected potential (kWh/year)", f"{tot_kwh_sel:,.0f}")
    st.write("Distribution of solar_score among selected buildings:")
    if not selected_gdf.empty:
        st.bar_chart(selected_gdf["solar_score"].value_counts(bins=10).sort_index())

    # Save geometry results to Supabase
    if st.button("Save suitability results to Supabase"):
        try:
            payload = {
                "city": city,
                "geojson": json.loads(buildings.to_json())
            }
            insert = supabase.table("building_suitability").insert(payload).execute()
            st.success("Saved building_suitability record to Supabase.")
        except Exception as e:
            st.error(f"Could not save to Supabase: {e}")

    st.success("Model run complete.")